# MEPI 2016
##### La pobreza energética se puede definir como la condición de carencia en el acceso a servicios energéticos mímimos de calidad que impacta de manera negativa la vida de las personas que la padecen. Se han propuesto varios índices para medir la pobreza energética de entre los cuales uno de los más utilizados ha sido el índice Multidimensional de Pobreza energética o MEPI (Multidimensional Energy Poverty Index). Este índice se compone de 5 dimensiones y 6 variables que toman en cuenta distintos servicios energéticos:  
* Tipo de combustible para cocción de alimentos
* Se usa un combustible distinto a gas o electricidad para cocinar
* Se tiene Acceso a electricidad
* Se tiene refrigerador
* Se cuenta con televisión o radio
* Se tiene un línea de teléfono fija o móvil

##### Cada una de las variables que se considera tiene un peso que se cuantifica para determinar si una vivienda se encuentra en pobreza energética, la imagen siguiente resume como se cuantifica cada varaible. 
![variablesMEPI.png](MEPI_categorias.PNG)

## Pregunta a resolver
##### La pregunta resolver en este trabajo es ¿Cómo ha variado el índice de pobreza energética de México a lo largo del tiempo? 

## Preparar datos
##### Los datos necesarios para calcular el índice de pobreza energética de acuerdo a la metodología MEPI se obtienen de la Encuenta Nacional de INgresos y GAstos en el Hogar elaborada por el INEGI cada 2 años. El tamaño de la muestra de viviendas se determina de acuerdo al ingreso trimestral promedio con un nivel de confianza del 90%.

##### Las variables de la ENIGH que pueden ser utilizadas para realizar la metodología MEPI son :
|Variable MEPI   | Conjunto datos | Columna     |
|----------------|----------------|-------------|
|Tipo combustible| Viviendas      | [25] combustible |
|Cocina en estufa| Viviendas      | [25] combustible (1,2,6) |
|Acceso electricidad| Viviendas      | [22] disp_Elec |
|Electrodomésticos| Hogares     | [78] num_refri |
|Entretenimiento/educacion| Hogares     | [62]num_radio, [64]num_tva, [66]num_tvd  |
|Comunicación| Hogares     | [36]telefono, [37]celular |


Se consideran estos dos conjuntos de datos (Viviendas y Hogares) y se procede a la limpieza de los datos


## Integridad de los datos 

Los datos utilizados para este trabajo fueron los proporcionados en la Encuesta Nacional de Ingresos y Gastos en el Hogar, recopilados por el Instituto Nacional de Estadística y Geografia en el 2016. El diseño de la muestra se determinó de forma probabilística, de manera que los resultados se pueden generalizar a toda la población del país. La unidad de observación de la muestra es el hogar y se dividió el total de hogares en México por tipos de asentamiento (rural, urbano alto y completamente urbano) y también se clasificaron las viviendas por características sociodemográficas, de esta manera se formaron 693 unidades primarias de muestreo en todo el país. El tamaño de la muestra se determinó con un nivel de confianza del 90% y se obtuvo que el tamaño mínimo de la muestra debe ser de 2000 viviendas por estado, sin embargo en algunos estados se encuestaron más viviendas y se tuvo un total de 81,515 viviendas encuestadas en todo el país. Los datos personales proporcionados son tratados y resguardados bajo las medidas de seguridad que garantizan su confiabilidad, con base en principios de licitud, consentimiento, calidad, información y responsabilidad establecidos en la Ley General de Protección. 



## Limpieza de datos
1. Se tomaron las columnas que se utilizarán de cada uno de los conjuntos de datos y se crearon dos nuevos conjuntos de datose en los que se ponen los numeros de folio de cada hogar y las columnas que se utilizaran. 

2. En estos arreglos hay diferente numero de filas, esto se debe a que puede hay más hogares que viviendas, 70311 hogares y 69169 viviendas. Se optó por considerar el número de viviendas para estar en acuerdo con el estudio realizado por Santillan.  

3. Se utilizó una consulta de sql para encontrar el número de viviendas que tienen más de un hogar, se encontró que 997 viviendas tienen más de un hogar. La pregunta es ¿que hacer con estos hogares? y ¿qué datos tomar?
``` sql
CREATE TEMPORARY TABLE temp_table AS 
  SELECT COUNT(folioviv) AS no_hogares, folioviv
  FROM my-project0-390719.energy_poverty.hogares  
  GROUP BY folioviv ;

SELECT * FROM temp_table
WHERE no_hogares > 1 
```

4. Despues de revisar la tabla de hogares que contiene las variables num_refri, num_tva, num_tvd, num_radio, las cuales son variables numéricas que se pueden sumar para obtener el total de aparatos en la vivienda. Para esto se utilizó la siguiente query: 
``` sql
SELECT folioviv, COUNT(folioviv) AS no_hogares, SUM(num_refri) AS num_refri2, SUM(num_radio) AS num_radio2, SUM(num_tva+num_tvd) AS no_tvs
FROM my-project0-390719.energy_poverty.hogares2
GROUP BY folioviv
```
5. Pero como lo que importa es que si tengan estos electrodomésticos y no cuantos tienen, se procedió a crear una tabla nueva en la que se cambio a variables boolenas, donde 1 es que NO tiene el electrodomestico y 0 es que SI lo tiene, quedaron 2 variables, refri, radio-tv. Esto se hizo a través de la siguiente query y posteriormente se guardó esta tabla con el nombre de electrodom3. ES IMPORTANTE notar que en el analisis MEPI la falta del electrodoméstico es el 1 y el si tenerlo es el 0.  

``` sql
WITH a AS (
SELECT folioviv, COUNT(folioviv) AS no_hogares, SUM(num_refri) AS num_refri2, SUM(num_radio) AS num_radio2, SUM(num_tva+num_tvd) AS no_tvs
FROM my-project0-390719.energy_poverty.hogares2
GROUP BY folioviv
)
#Nota: se asigna un valor de 0 a las viviendas que si tienen refri y un valor de 1 a las que no, y tambien para la variable radio-tv, si los radios o la tv es mayor a 1, la variable toma el valor de 0 y sino toma el valor de 1. 
SELECT folioviv, IF(a.num_refri2>=1, 0, 1) AS refri, IF(a.num_radio2>=1 OR a.no_tvs>=1,0, 1) AS radio_tv 
FROM a
```

6. Por otra parte, las variables telefono y celular so variables booleanas (de sí o no), así que con que alguno de los hogares tenga el valor de sí (1), se considerará un sí para esa vivienda. Para ello se creó una nueva columna que englobe estas dos columnas utilizando el siguiente query. Los resultados se guardaron en la tabla telefono2.  
``` sql
WITH x AS 
( SELECT folioviv, telefono, celular, IF(telefono = 1 OR celular=1,1,0 ) AS telefono2
  FROM my-project0-390719.energy_poverty.hogares2  ),
y AS 
( SELECT folioviv, SUM(telefono2) AS telefono3, COUNT(folioviv) AS no_hogares
  FROM x
  GROUP BY folioviv), 
z AS 
( SELECT folioviv, IF (telefono3>= 1, 0, 1) AS telefono4 FROM y)
SELECT folioviv, telefono4 FROM z
```

7. El siguiente paso es juntar las 3 variables de electrodomesticos con la de telefonos en una sola tabla donde el índice sea el número de vivienda, el resultado se guardó en la tabla var_vivienda3
``` sql
SELECT electrodomesticos.folioviv, refri,radio_tv, telefono4 
FROM my-project0-390719.energy_poverty.electrodom3 AS electrodomesticos
INNER JOIN my-project0-390719.energy_poverty.telefono2 AS telefono
ON electrodomesticos.folioviv = telefono.folioviv
```

8. Ahora se debe tomar la tabla de vivienda y pasar las variable de combustible, disp_elect y estufa_chi a booleana. La variable de combustible tiene  6 opciones: 1)leña, 2)carbon, 3) gas de tanque, 4) gas natural de tuberia, 5)electricidad, 6) otro. Como la pregunta que se queire responder respectdo a combustible es si el combustible utilizado es diferente a electricidad, gas natural, queroseno o biogas? entonces si la variable combustible tiene el valor 1)leña o 2) carbon o 6) otro, etonces la variable debe valer 1, si es diferente debe valer 0.  Me sorprendió que 12225 viviendas cocinan con leña, carbon u otro. 

9. Por otra parte, la variable disp_electrica tiene 5 opciones: 1)servicio público, 2)de planta particular, 3) de panel solar, 4) de otra fuente, 5)no tiene luz. Entonces para pasar a booleana de si o no, se considera el valor 5) como 1 es decir no tiene, y el resto de valores como 0, es decir si tiene. Los pasos 8 y 9 se llevaron a cabo a través del siguiente query y se exportan los resultados como viviendas4: 
``` sql
WITH a AS (
#se crearon las condiciones si combustibles=1,2,6 es 1 porque cocina con leña, carbon etc, si la disp_elect=5 es 1 porque no tienen electricidad   
SELECT folioviv, IF(combustible = 1 OR combustible=2 OR combustible=6,1,0) AS combustible2, IF(disp_elect = 5,1,0) AS electricidad 
FROM my-project0-390719.energy_poverty.viviendas  
)
SELECT folioviv, combustible2, electricidad
#SELECT folioviv, disp_elect
FROM  a  
#WHERE disp_elect = 1
#WHERE combustible =1 OR combustible=2 OR combustible=6
```

10. Despues de escribirle a Oss, me comentó que las variables que ellos usaron como combustible "sucio" fueron solamente leña y carbon y no otro, por lo que se cambia el query.  

11. Luego, la variable estufa_chi tiene 3 opciones posibles: vacio, 1 y 2, donde 1 significa que la vivienda si tiene chimenea para desalojar el humo, lo cual es bueno y se considera como un 0 para el índice, mientras que 2 significa que no tiene, lo cual es malo y se considera como un 1 para el índice. Y la celda vacia significa que no cocinan con un combustible sucio, por lo que la variable tambien toma el valor 0. Un reto más es pasar la variable estufa_chi a enteros. La query queda como a continuación y se renombra la tabla como viviendas5
``` sql
WITH a AS (
#se crearon las condiciones si combustibles=1,2 es 1 porque cocina con leña, carbon etc, si la disp_elect=5 es 1 porque no tienen electricidad   
SELECT folioviv, IF(combustible = 1 OR combustible=2,1,0) AS combustible2, IF(disp_elect = 5,1,0) AS electricidad, 
IF(estufa_chi = "1" OR estufa_chi=" ",0,1) AS estufa3
FROM my-project0-390719.energy_poverty.viviendas  
)
SELECT folioviv, combustible2, electricidad, estufa3
#SELECT folioviv, disp_elect
FROM  a  
#WHERE estufa3 = 1
```

~~9. Y la variable estufa_chi que se refiere a si hay presencia de chimenea pra desalojar el humo al cocinar: 1) si, 2) no. Mientras que la variable num_estuf en la tabla hogares2 se refiere al número de estufas de gas o eléctricas con las que cuenta el hogar, por lo que para responder a la pregunta de si se cocina es estufa o fuego abierto (chimenea) se utilizan las siguientes suposiciones: si el número de estufas de gas o eléctricas es 0, es posible que usen fogon, 7496 viviendas estan en esta situacion. Por otro lado, la variable estufa_chi que indica si hay chimenea en la vivienda tiene el problema de que hay muchas filas vacias, 57471 filas vacias, 3377 filas donde la respuesta es si (1), 8321 donde la respuesta es no (2) y si la vivienda tiene chimenea entonces si el departamento tiene chimenea. Lo que se hizo fue tomar la variable num_estufas y convertirla a una variable booleana donde 1 es si el numero de estufas de gas o electricas es 0, y se le asigna el valor 0 si el número es mayor o igual a 1, luego se agruparon estos datos por vivienda y se guardó esta variable en la tabla estufa. Se usa la siguiente query:~~

``` sql
WITH a AS (
SELECT folioviv, SUM(num_estuf) AS tot_estuf
FROM my-project0-390719.energy_poverty.hogares2
GROUP BY folioviv )
SELECT folioviv, IF(a.tot_estuf=0,1,0) AS estufa
FROM a
```

~~10. Se tienen que unir las tablas "estufa" y "viviendas5", para lo cual se usa la siguiente query, y se guarda en la tabla "estu-comb".~~  
``` sql
SELECT viviendas4.folioviv, combustible2,electricidad, estufa
FROM my-project0-390719.energy_poverty.viviendas4 AS viviendas4
INNER JOIN my-project0-390719.energy_poverty.estufa AS estufa1
ON viviendas4.folioviv = estufa1.folioviv
```
11. LUego se une esta tabla "estu-comb" con la tabla "var_vivienda3" usando la siguiente query y se guardo la tabla con el nombre de "Matriz1"
``` sql
SELECT from_vivienda.folioviv, combustible2, estufa, electricidad, refri, radio_tv, telefono4
FROM my-project0-390719.energy_poverty.estu_comb AS from_vivienda
INNER JOIN my-project0-390719.energy_poverty.var_vivienda3 AS from_hogares
ON from_vivienda.folioviv = from_hogares.folioviv
``` 
De esta forma, se construyó una matriz que tiene 7 columnas y 69,169 filas. Las columnas son: folioviv, combustible, electricidad, estufa, refri, radio_tv, telefono4.  Para crear esta tabla se utilizó la siguiente query: 



# Análisis MEPI
Cada fila de la matriz es una vivienda, si la vivienda cumple con las condicioens de privación previamente definidas, entonces esa variable toma el valor del vector de pesos definido. En este caso se usa el vector de pesos utilizado por Nussbaumer y por Santillan para poder replicar el análisis realizado por ellos. En este vector de pesos las variables toman los siguientes valores: 
- a) Modern cooking fuel = 0.2 
- b) indoor pollution = 0.2
- c) electricity access = 0.2
- d) no tiene refri = 0.13
- e) no tiene radio o television = 0.13
- f) no tiene telefono = 0.13 

Despues de multiplicar cada variable por su valor del vector de pesos se suma el tota de los pesos para cada vivienda, lo que da un valor al que se le asigna la letra $c(i)$. Si este valor $c(i)$ es mayor a un valor $k$ que este caso se asigna de $k=0.3$ este valor de $k$ representa a las personas en pobreza energética extrama según Santillan. Mientras que para calcular el MEPI normal se usó como condición para identificar a una vivienda como pobre energéticamente el que no contarán con alguno de los servicios energéticos. 

El MEPI se calcula como el producto de las variables incidencia ($H$) e intensidad ($A$) las cuales se expresan como $H=q/n$ donde $q$ es el número de personas identficiadas como pobres energéticamente y $n$ es el total de viviendas/personas.  POr otra parte la intensidad se calcula como $A= \frac{\sum_{i=1}^n c(i) }{q}$. Y $MEPI = A*H$. 

1. SE manipuló la tabla "Matriz1" para multiplicar cada columna por su respectivo valor de peso del vector de pesos

In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

## Multiplicacion de las columnas por sus pesos 

In [5]:
data0 = pd.read_csv("Matriz2.csv")
data0.combustible2 = data0.combustible2.multiply(0.2)
data0.estufa3 = data0.estufa3.multiply(0.2)
data0.electricidad = data0.electricidad.multiply(0.2)
data0.refri = data0.refri.multiply(0.13)
data0.radio_tv = data0.radio_tv.multiply(0.13)
data0.telefono4 = data0.telefono4.multiply(0.13)
data0["c"] = data0["combustible2"] + data0["estufa3"] + data0["electricidad"]+ data0["refri"] + data0["radio_tv"]+data0["telefono4"]    
#data0.c = data0.c.astype(float)

In [7]:
data0[40:50]

,folioviv,combustible2,estufa3,electricidad,refri,radio_tv,telefono4,c
40,100076305,0.0,0.0,0.0,0.0,0.0,0.00,0.00
41,100076306,0.0,0.0,0.0,0.0,0.0,0.00,0.00
42,100079801,0.0,0.0,0.0,0.0,0.0,0.00,0.00
43,100079802,0.0,0.0,0.0,0.0,0.0,0.00,0.00
44,100079803,0.0,0.0,0.0,0.0,0.0,0.00,0.00
45,100079804,0.0,0.0,0.0,0.0,0.0,0.00,0.00
46,100079806,0.0,0.0,0.0,0.0,0.0,0.13,0.13
47,100101401,0.0,0.0,0.0,0.0,0.0,0.00,0.00
48,100101402,0.0,0.0,0.0,0.0,0.0,0.00,0.00
49,100101403,0.0,0.0,0.0,0.0,0.0,0.13,0.13


In [7]:
len(data0)

69169

## Conteo de personas que cumplen con $c(i) > k$, calculo de intensidad y MEPI

In [9]:
count = 0
k = 0.3
suma = 0

for a in range(len(data0)):
    if (data0["c"][a] > k ) :
          count = count + 1
          suma =suma + data0["c"][a]
print(count) 
H = count/len(data0)
A = suma/count
MEPI = A*H

print("Porcentaje hogares pobres (H) ", H )          
print("Suma de c=",suma)
print("La intesidad es=",A)
print("el MEPI = ", MEPI)

10502
Porcentaje hogares pobres (H)  0.15183102256791337
Suma de c= 5226.449999999721
La intesidad es= 0.49766235002853937
el MEPI =  0.07556058349838396


## Export to csv arreglo foliioviv-c(i)

In [11]:
data0.to_csv('2016c.csv', columns=['folioviv','c'], index=False) 

In [13]:
type(data0["folioviv"][0])
print(data0["folioviv"][0])

100003801


# SIGUIENTES PASOS 4/3/2024
- ~~ESCRIBIR A OSS PARA VER SI ME PUEDE ACLARAR COMO SE CONSIDERARON LAS VARIABLES~~
- ~~EXPORTAR CSV DE FOLIOVIV-c para DIVIDIRLO EN SQL POR ESTADO~~
- ~~DIVIDIR LAS VIVIENDAS POR ESTADO PARA CALCULAR UN VALOR DE MEPI POR ENTIDAD Y~~
- CREAR UN MAPA INTERACTIVO EN TABLEU EN QUE SE PUEDAN CAMBIAR LOS AÑOS 
- DESCARGAR DATOS DE LOS OTROS AÑOS 
- LIMPIEZA DE DATOS DE TODOS LOS AÑOS
